# 空调健康评估模型
## 1 模型运行环境概述
模型运行在工业互联网平台。数据从现场PLC系统采集的空调的冷水机组、风机、水泵的数据以及加装的振动传感器数据到物联网平台。
物联网平台将数据转发到数据总线，再通过数据总线为所有后台模型业务提供服务。
## 2 空调背景知识
空调的主要部件包含：冷水机组、水泵、冷却塔、空气处理单元。如下图所示的空气处理单元有AHU和FCU之分，前者常见于大型中央空调、后者是家用室内机。FCU不像AHU有复杂的管道和风机，具备新风换气功能，它只对封闭空间的气体循环制冷或制热。在本项目中，AHU加热采用蒸汽加热。

![image.png](images/ac-overview.png)
![image.png](images/ahu.jpg)


## 3 振动信号分析建模

旋转机械分析主要关注与转速及其谐波相关的信号。阶数是转速的谐波。阶次分析对许多类型的旋转机器都至关重要。 

- 分析振动频谱和PSD，对稳定工况进行分析。

- 阶次分析，对启动、停止等变工况信号进行分析。

除FFT 分析、阶次跟踪分析，还有扭转振动、轨道分析和倍频分析，[参考成熟软件](https://dewesoft.com/applications/vibration-analysis/rotational-and-torsional-vibration)。



## 4 空调健康微服务说明

### 微服务API文档
微服务发布在多核国产服务器。
- Comboindicator，该服务按输入参数调用模型服务，并推送SOH、SOC指标；
- Kill，该API将停止服务的运行，当需要在Jupyter中重启微服务，需要调用该接口，然后再启动Jupyter的Kernel，运行模型代码。

In [2]:
import pandas as pd

ns=['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
pd.read_table('./data/2003.10.22.12.06.24', names=ns)


,c1,c2,c3,c4,c5,c6,c7,c8
0,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142
1,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122
2,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071
3,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078
4,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066
...,...,...,...,...,...,...,...,...
20475,-0.022,-0.142,0.085,-0.078,-0.200,-0.198,-0.134,-0.093
20476,0.024,-0.156,0.042,-0.039,-0.159,-0.071,-0.227,-0.132
20477,-0.193,-0.303,-0.071,-0.125,-0.237,-0.251,-0.151,0.007
20478,-0.115,0.076,-0.078,-0.332,-0.027,-0.002,-0.151,-0.005


In [3]:
# Copyright 2021 The CASICloud Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
# pylint: disable=invalid-name
# pylint: disable=missing-docstring
"""旋转机械健康管理模型.

参考:
  - [1]Neupane D, Seok J. Bearing fault detection and diagnosis using case western reserve university
  dataset with deep learning approaches: A review[J]. IEEE Access, 2020, 8: 93155-93178.
"""

# Author: Awen <26896225@qq.com>
# License: MIT

import json
import logging
import multiprocessing as mul
import phm.modules.mqttworker as mqttworker
import phm.modules.restworker as restworker
import phm.phm as phm
from phm import BENCHPATH


# setup and cache mqtt connection
def setup_mqtt(process, param):
    hst = param['host']
    tk = param['token']
    pt = param['port']

    (chB, chA) = mul.Pipe()
    if process:
        process.terminate()
        process.join()
    process = mul.Process(target=mqttworker.proc_mqtt, args=(chB, tk, hst, pt))
    process.start()
    process.chanel = lambda: None
    setattr(process.chanel, 'channel', chA)
    return process, chA


# 计算指标并推送
def compute_publish(process, channel, param):
    try:
        if process:
            retidx = phm.calculate_mds_indicator(param, BENCHPATH)
            if retidx:
                for item in retidx:
                    mds = {"ts": item['ts'], "values": {"MDS": item['MDS']}}
                    channel.send(json.dumps(mds))
        else:
            logging.info('Mqtt connectoin should be set and cache firstly.')
    except BrokenPipeError as be:
        logging.error(f'compute_publish to mqtt error.')
    return


if __name__ == "__main__":
    # prepare communication channels
    (endRE_B, endRE_A) = mul.Pipe()
    # init restful service
    rest_p = mul.Process(target=restworker.proc_rest, args=(endRE_B,))
    rest_p.start()
    # prepare mqtt service
    mqtt_p = None
    endMQ_A = None
    while True:
        data = endRE_A.recv()
        obj = json.loads(data)
        cmd = obj['command']
        if cmd == 'comboindicator':  # 根据观测值计算健康评估指标
            reconnect = obj['reconnectmqtt']
            if reconnect:
                (mqtt_p, endMQ_A) = setup_mqtt(mqtt_p, obj)
            compute_publish(mqtt_p, endMQ_A, obj)
        elif cmd == 'setupmqtt':  # 建立MQTT通道
            (mqtt_p, endMQ_A) = setup_mqtt(mqtt_p, obj)
            logging.info('Mqtt channel is set.', mqtt_p, endMQ_A)
        elif cmd == 'kill':
            logging.info(obj)
            break
        else:  # 异常情况
            if mqtt_p:
                endMQ_A.send('stop')
                mqtt_p.join()
                mqtt_p = None
            break

    logging.info('Main loop terminated, service exit.')


### 为了运行本模型服务并调测微服务，请首先确保点击菜单Run，Run All Cells。

[在线调测](http://127.0.0.1:29054/docs)

[微服务调用文档](http://127.0.0.1:29054/redoc)